In [32]:
import gzip
from collections import OrderedDict

In [33]:
gtf_file = '/scratch/research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.gz'

In [47]:
class gtf_line():
    def __init__(self, l, info_delimiter=";", info_field_delimiter = '='):
        self.seqid, self.source, self.type, self.start, self.end,\
        self.score, self.strand, self.phase, self.attribs = l.split('\t')
        self.start = int(self.start)
        self.end = int(self.end)
    def attribute_dict(self, attributes, info_delimiter="; ", info_field_delimiter = ' '): ###this is fragile
        d = OrderedDict()
        attributes = attributes.split(info_delimiter)
        for kv in attributes:
            k,v = kv.split()
            v = v.strip(';')
            v = v.strip('"')
            d[k]=v
        return d

In [64]:
from Bio import SeqIO

ref_dict = SeqIO.to_dict(SeqIO.parse(gzip.open('/scratch/research/references/hsapiens/hg38/hg38.fa.gz', 'rt'), 'fasta'))

In [ ]:
bc =0
last_transcript = []
current_ch = ''
intergenic_outfile = open("intergenic.fasta", 'w')
for l in gzip.open(gtf_file, "r"):
    l = l.decode('utf8').strip()
    g = gtf_line(l)
    if g.type != 'transcript':continue
    if g.seqid != current_ch:
        print(g.seqid)
        if len(g.seqid) >5:
            continue
        last_transcript = g
        current_ch = g.seqid
        print(current_ch)
        continue
    #if this new transcript doesn't overlap the last output the intergenic region
    if g.start > last_transcript.end:
        # new non overlapping
        # output intergenic
        seq = ref_dict[g.seqid].seq[int(last_transcript.end)-1: int(g.start)]
        intergenic_outfile.write(">{c}:{s}-{e}\n{seq}\n".format(c=g.seqid,s=last_transcript.end,e=g.start,seq=str(seq)))
        # set last
    last_transcript = g
    

In [78]:
intergenic_outfile.close()

In [81]:
!grep ">" intergenic.fasta |wc -l

30452


In [ ]:
bc =0
last_cds = None
current_ch = ''
cds_outfile = open("cds.fasta", 'w')
for l in gzip.open(gtf_file, "r"):
    l = l.decode('utf8').strip()
    g = gtf_line(l)
    if g.type != 'CDS':continue
    if g.seqid != current_ch:
        if len(g.seqid) >5:
            continue
        print(g.seqid)
        last_cds = g
        current_ch = g.seqid
        print(current_ch)
        continue
    #if this new transcript doesn't overlap the last output the intergenic region
    if g.start > last_cds.end:
        # new non overlapping
        # output intergenic
        seq = ref_dict[g.seqid].seq[g.start-1: g.end]
        cds_outfile.write(">{c}:{s}-{e}\n{seq}\n".format(c=g.seqid,s=g.start,e=g.end,seq=str(seq)))
        # set last
    last_cds = g
    

In [85]:
cds_outfile.close()

In [84]:
!grep ">" cds.fasta |head

>chr1:358070-358183
>chr1:373144-373323
>chr1:379769-379870
>chr1:399041-399100
>chr1:365565-365692
>chr1:373144-373323
>chr1:379769-379870
>chr1:399041-399100
>chr1:450743-451678
>chr1:586842-586955
grep: write error


In [57]:
!zcat /scratch/research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.gz |head -n 100

chr1	ncbiRefSeq	transcript	11874	14409	.	+	.	gene_id "DDX11L1"; transcript_id "NR_046018.2";  gene_name "DDX11L1";
chr1	ncbiRefSeq	exon	11874	12227	.	+	.	gene_id "DDX11L1"; transcript_id "NR_046018.2"; exon_number "1"; exon_id "NR_046018.2.1"; gene_name "DDX11L1";
chr1	ncbiRefSeq	exon	12613	12721	.	+	.	gene_id "DDX11L1"; transcript_id "NR_046018.2"; exon_number "2"; exon_id "NR_046018.2.2"; gene_name "DDX11L1";
chr1	ncbiRefSeq	exon	13221	14409	.	+	.	gene_id "DDX11L1"; transcript_id "NR_046018.2"; exon_number "3"; exon_id "NR_046018.2.3"; gene_name "DDX11L1";
chr1	ncbiRefSeq	transcript	14362	29370	.	-	.	gene_id "WASH7P"; transcript_id "NR_024540.1";  gene_name "WASH7P";
chr1	ncbiRefSeq	exon	14362	14829	.	-	.	gene_id "WASH7P"; transcript_id "NR_024540.1"; exon_number "1"; exon_id "NR_024540.1.1"; gene_name "WASH7P";
chr1	ncbiRefSeq	exon	14970	15038	.	-	.	gene_id "WASH7P"; transcript_id "NR_024540.1"; exon_number "2"; exon_id "NR_024540.1.2"; gene_name "WASH7P";
chr1	ncbiRefSeq	exon	15796